# Continuous Optimisation HW2

In [ ]:
# Imports
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from tqdm import tqdm
from scipy.io import loadmat


data = loadmat('data.mat')
print(data.keys())
sigma = float(data['sigma'])

'''
K = data['K']
P = data['P']
X0 = data['X0']
d = data['d']
delta_0 = data['delta_0']
delta_bar = data['delta_bar']
n = data['n']
sigma = data['sigma']
y = data['y']
'''

sigmasquared = sigma**2


# Question 1
Implement phi(x, P), bigphi(X, P), f(X, P, y).

In [ ]:
# Question 1
#TODO Vectorise all the functions

def h(x: np.ndarray) -> float:
    """
    Gaussian filter

    :x: np.ndarray[(1, 2)]
    :returns: float
    """
    return np.e**(-np.inner(x, x)/sigmasquared)  # Always take sigma = 0.1


def phi(x,P):
    """
    Calculate contribution of each 'true' star to observed image
    
    :x: np.ndarray[(1, 2)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 1)]
    """
    
    return (np.e**((-1/sigmasquared)*np.sum((P.T-x.T)**2,axis=1))).reshape(-1, 1)



def bigphi(X, P):
    """
    Calculate image observed, based on K-star positions X
    
    :X: np.ndarray[(2, K)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 1)]
    """
    global K, n
    #non-vectorised code just in case we need it
    bigphi = np.zeros((n**2, 1))
    for i in range(K):
         bigphi += phi(X[2*i:2*i+2], P)
    #return phi(X, P)
    return bigphi


def  f(X, P, y):
    """
    Calculate squared error of estimate bigphi(X)
    
    :X: np.ndarray[(2, K)]
    :P: np.ndarray[(2, n**2)]
    :y: np.ndarray[(n**2, 1)]
    :returns: float
    """
    global K, n
    return (1/(2*n**2)) * np.linalg.norm(bigphi(X, P)-y)**2

# Question 2

We see that $f$ is not convex. There are clear local maxima which cannot occur if $f$ were convex.

In [ ]:
K = 1
n = 2
true_positions = np.array([[0], [0]])
positions = np.array([[0.5, 0.5], [-0.5, 0.5], [-0.5, -0.5], [0.5, -0.5]]).T
y = np.array([[0], [0], [1], [0]])
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

x_vals = np.linspace(-1, 1, 100)
y_vals = np.linspace(-1, 1, 100)
x_vals, y_vals = np.meshgrid(x_vals, y_vals)
grid = np.array([x_vals, y_vals]).reshape((2, 100**2))
z_vals = np.zeros((1, 10000))
for i in range(10000):
    z_vals[0, i] = f(grid[:, i].reshape((2, 1)), positions, y)
z_vals = z_vals.reshape((100, 100))
ax.scatter(x_vals, y_vals, z_vals)
plt.show()
plt.show()

# Question 6.1

In [ ]:


#computes value of cell i_0 of jacobian of phi
def d_phi_i_0(x,p_i):
    return (2/sigmasquared)*(p_i[0]-x[0])*np.e**((-1/sigmasquared)*np.inner(p_i-x,p_i-x))
#computes value of cell i_1 of jacobian of phi
def d_phi_i_1(x,p_i):
    return (2/sigmasquared)*(p_i[1]-x[1])*np.e**((-1/sigmasquared)*np.inner(p_i-x,p_i-x))

'''
#compute jacobion of small phi
def d_phi(x,P):
    d_phi = np.zeros((n**2,2))
    for i in range(n**2):
        P_i = np.array([P[0][i],P[1][i]])
        d_phi[i][0] = d_phi_i_0(x,P_i)
        d_phi[i][1] = d_phi_i_1(x,P_i)
    return d_phi
'''
    

# Compute jacobian of small phi
def d_phi(x, P):
    """
    :x: np.ndarray[(1, 2)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 2)]
    """
    # Subtract x from each column of P
    diff = P.T - x
    # Compute the squared differences
    squared_diff = np.sum(diff**2, axis=1)
    # Compute the exponential term for all elements at once
    exp_term = np.exp((-1 / sigmasquared) * squared_diff)
    # Compute the values for d_phi_i_0 and d_phi_i_1 using vectorized operations
    d_phi_0 = (2 / sigmasquared) * diff[:, 0] * exp_term
    d_phi_1 = (2 / sigmasquared) * diff[:, 1] * exp_term
    # Stack the results to form the final array
    d_phi = np.stack((d_phi_0, d_phi_1), axis=1)
    return d_phi

#compute jacobian of big phi
def d_big_phi(X,P):
    global K, n
    d_big_phi = np.zeros((n**2,2*K))
    for i in range(K):
        d_big_phi[0:n**2,2*i:2*i+2] = d_phi(X[2*i:2*i+2].T,P)
    return d_big_phi

#compute jacobian of f
def d_f(X,P):
    print(y.shape)
    return (1/n**2)*(bigphi(X,P)-y).T@d_big_phi(X,P)

#compute gradient of f
def grad_f(X,P):

    '''
    returns: np.ndarray[(n**2, 2*K)]
    '''

    return d_f(X,P).T

# Question 6.2

In [ ]:
#checking gradient is correct numerically
K = int(data['K'])
P = data['P']
X0 = data['X0']
d = int(data['d'])
delta_0 = data['delta_0']
delta_bar = data['delta_bar']
n = int(data['n'])
sigma = float(data['sigma'])
y = data['y'].flatten(order='F').reshape(n**2,1)

#check that the gradient is correct using
# f(x+tv)= f(x) + t<v,grad_f(x)> + O(t^2)

# Generate a random point and a random direction
theta = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)
v = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)
v = v / np.linalg.norm(v)

## Check the gradient 
def checkgradient(f,grad_f, theta,v):
    #logspace of t values
    t=np.logspace(-8, 0, num=100)
    #intialise error to 0
    error = np.zeros_like(t)
    #pre-calculae f_lambda and f_lambda_grad to use in for loop
    f_lambda = f(theta,P,y)
    f_lambda_grad = grad_f(theta,P)
    #compute the error at each t
    for i in tqdm(range(100)):
        error[i] = np.abs( f(theta+(t[i]*v),P,y)-f_lambda-(t[i]*v.T@f_lambda_grad) )
        
    #plot the graph of error vs t
    plt.loglog(t,error)
    plt.xlabel('t (log scale)')
    plt.ylabel('Error (log scale)')
    plt.title('Plot of t vs Error')
    plt.grid()
    plt.show
checkgradient(f,grad_f,theta,v)

In [ ]:
#debugging

# Question 7

## Computing the Hessian 

First we re-derive the gradient of f:

$$ f(x) = \frac{1}{2n^2}||\Phi(x) - y ||^2$$

$$ Df(x)[v] = \frac{1}{n^2}< \Phi(x) - y,D \Phi(x)[v] > = \frac{1}{n^2}<(D\Phi(x))^*[\Phi(x) - y],v> $$

$$ \nabla f(x) = \frac{(D\Phi(x))^*[\Phi(x) - y]}{n^2} $$

Then, using the above calculations, we can calculate the hessian:

$$ \frac{1}{n^2}[D((D\Phi(x))^*[\Phi(x) - y])[v]] = \frac{1}{n^2}[D((D\Phi(x))^*[v])[\Phi(x) - y] + (D\Phi(x))^*[D\Phi(x)[v]]]$$


$$ \nabla^2 f(x)[v] = \frac{1}{n^2}[(D(D\Phi(x))^*[v])[\Phi(x)-y]+D\Phi(x)^*[D\Phi(x)[v]]] $$

The implicit reason why we are calculating the hessian with respect to a direction v is because it is computationally much cheaper than to calculate the full hessian which means calculating all 2nd order partial derivatives of f.



In [ ]:
def d_d_phi(x, P, v):
    """
    Compute the directional derivative of the Jacobian of phi at x in the direction v.

    :param x: np.ndarray[(1, 2)] "position of a stat"
    :param P: np.ndarray[(2, n**2)] "pixel positions"
    :param v: np.ndarray[(1, 2)] "vector direction of change in star position"
    :returns: np.ndarray[(n**2, 2)]
    """
    diff = P.T - x
    squared_diff = np.sum(diff**2, axis=1)
    exp_term = np.exp((-1 / sigmasquared) * squared_diff)
    
    # Compute the directional derivatives of d_phi_i_0 and d_phi_i_1
    d_d_phi_0 = (2 / sigmasquared**2) * (diff[:, 0] * exp_term * np.inner(diff[:, 0], v))
    d_d_phi_1 = (2 / sigmasquared**2) * (diff[:, 1] * exp_term * np.inner(diff[:, 1], v))
    
    d_d_phi = np.stack((d_d_phi_0, d_d_phi_1), axis=1) 
    return d_d_phi

def hessian_f(X,v):
    """
    Compute the Hessian of f at X in the direction v.
    
    :param X: np.ndarray[(2, K)] "position of K stars"
    :param P: np.ndarray[(2, n**2)] "pixel positions"
    :param y: np.ndarray[(n**2, 1)] "actual image detected, y = \Phi(X_true)" remember we want to find X_true
    :param v: np.ndarray[(2, K)] "direction at which hessian is taken" 
    :returns: np.ndarray[(n**2, 2*K)]
    """

    # Compute intermediate terms
    phi_X = bigphi(X, P) - y  # (n**2, 1)
    d_big_phi_X = d_big_phi(X, P)  # (n**2, 2*K)
    d_big_phi_X_v = d_big_phi(X, P) @ v.T  # directional derivative of big_phi in direction v

    # Compute each term in the Hessian formula

    term1 = d_d_phi(X, P, v).T @ phi_X 
    term2 = d_big_phi_X.T @ d_big_phi_X_v 

    hessian = (1 / n**2) * (term1 + term2)
    return hessian



# Question 8


In [ ]:
# Haven't debuged this part yet 

'''

# Generate random X and U
X = X0
d = X.shape[0] 

X_random = np.random.uniform(-0.5, 0.5, (d, K))  # Random X
U_random = np.random.uniform(-0.5, 0.5, (d, K))  # Random direction U
U_random /= np.linalg.norm(U_random)  # Normalize U

# Generate logarithmically spaced t values
t_values = np.logspace(-8, 0, 100)

# Compute gradient and Hessian at X
grad_f_X = grad_f(X_random, P)  # Gradient of f at X_random # error is appearing here, seems like something is wrong with phi

hessian_f_X = hessian_f(X_random, U_random)  # Hessian of f at X_random in direction U_random

# Function to compute the error e(t)
def compute_error(f, grad_f, hessian_f, X, U, t_values):
    f_X = f(X, P, y)  # Compute f(X)
    grad_f_X = grad_f(X, P)  # Compute gradient at X
    error = np.zeros_like(t_values)
    
    for i, t in enumerate(t_values):
        # Compute the directional term (t * <U, grad_f(X)>)
        grad_term = t * (U.T @ grad_f_X)  # Scalar
        
        # Compute the Hessian term (t^2 / 2 * <U, Hessian_f(X)[U]>)
        hessian_term = (t**2 / 2) * (U.T @ hessian_f @ U)  # Scalar
        
        # Compute f(X + tU)
        f_X_tU = f(X + t * U, P, y)  # f(X + tU)
        
        # Compute the error e(t)
        error[i] = np.abs(f_X_tU - f_X - grad_term - hessian_term)
    
    return error

# Compute error
error = compute_error(f, grad_f, hessian_f, X_random, U_random, t_values)

# Plot error vs t
plt.loglog(t_values, error)
plt.xlabel('t (log scale)')
plt.ylabel('Error (log scale)')
plt.title('Plot of t vs Error for Hessian check')
plt.grid(True)
plt.show()

'''